# Условия задачи:

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

Импортируем библиотеки которые нужны или могут понадобиться.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, cross_val_score

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

Прочитаем данные. Ниже описание столбцов.

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей)

In [ ]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
'-'*80, 'data_0', '-'*80, data_0, '-'*80, 'data_1', '-'*80, data_1, '-'*80, 'data_2', '-'*80, data_2

('--------------------------------------------------------------------------------',
 'data_0',
 '--------------------------------------------------------------------------------',
           id        f0        f1        f2     product
 0      txEyH  0.705745 -0.497823  1.221170  105.280062
 1      2acmU  1.334711 -0.340164  4.365080   73.037750
 2      409Wp  1.022732  0.151990  1.419926   85.265647
 3      iJLyR -0.032172  0.139033  2.978566  168.620776
 4      Xdl7t  1.988431  0.155413  4.751769  154.036647
 ...      ...       ...       ...       ...         ...
 99995  DLsed  0.971957  0.370953  6.075346  110.744026
 99996  QKivN  1.392429 -0.382606  1.273912  122.346843
 99997  3rnvd  1.029585  0.018787 -1.348308   64.375443
 99998  7kl59  0.998163 -0.528582  1.583869   74.040764
 99999  1CWhH  1.764754 -0.266417  5.722849  149.633246
 
 [100000 rows x 5 columns],
 '--------------------------------------------------------------------------------',
 'data_1',
 '-------------------

Видим, что данные наши уже были закодированы и масштабированы. Проверим наши датасеты функцией info

In [ ]:
print(data_0.info(), '='*80)
print(data_1.info(), '='*80)
print(data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None ================================================================================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None ================================================================================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         1000

Данные датасетов имеют нужные нам тип данных.

In [ ]:
print(data_0.isna().sum(), '='*80)
print(data_1.isna().sum(), '='*80)
print(data_2.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 ================================================================================
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 ================================================================================
id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Данные не имеют пропущенных значений. Проверим на дубликаты.

In [ ]:
print(data_0.duplicated().sum())
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())

0
0
0


Запишем в переменные признаки и целевой признак для каждого датасета. Удалим столбец id т.к. он не несет ни какой инфы для алгоритма.

In [ ]:
target_0 = data_0['product']
features_0 = data_0.drop(['id', 'product'], axis=1)

target_1 = data_1['product']
features_1 = data_1.drop(['id', 'product'], axis=1)

target_2 = data_2['product']
features_2 = data_2.drop(['id', 'product'], axis=1)

Данные готовы к работе.

## Обучение и проверка модели

Разбиваем признаки и целевой признак на выборки по регионам. stratify не используем т.к. решаем задачу регресии.

In [ ]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, random_state=12345, test_size=.25)

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, random_state=12345, test_size=.25)

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, random_state=12345, test_size=.25)

Проверим размер выборок.

In [ ]:
features_train = features_train_0, features_train_1, features_train_2
features_valid = features_valid_0, features_valid_1, features_valid_2

In [ ]:
for i in range(len(features_train)):
    print(features_train[i].shape, features_valid[i].shape)

(75000, 3) (25000, 3)
(75000, 3) (25000, 3)
(75000, 3) (25000, 3)


Обучаем модель и делаем предсказания на валидационной выборке.

Делаем модель для каждого региона и обучаем.

In [ ]:
model_0 = LinearRegression()
model_1 = LinearRegression()
model_2 = LinearRegression()

model_0.fit(features_train_0, target_train_0)
model_1.fit(features_train_1, target_train_1)
model_2.fit(features_train_2, target_train_2)

predictions_valid_0 = model_0.predict(features_valid_0)
predictions_valid_1 = model_1.predict(features_valid_1)
predictions_valid_2 = model_2.predict(features_valid_2)

Сохраняем предсказания по валид выборке

In [ ]:
prediction_0 = pd.Series(predictions_valid_0, index=target_valid_0.index)
prediction_1 = pd.Series(predictions_valid_1, index=target_valid_1.index)
prediction_2 = pd.Series(predictions_valid_2, index=target_valid_2.index)

Выводи на экран средний запас предсказанного сырья и RMSE модели.

In [ ]:
mse_0 = mean_squared_error(target_valid_0, predictions_valid_0)
rmse_0 = mse_0 ** 0.5
r2_0 = model_0.score(features_valid_0, target_valid_0)
print('Средний запас предсказанного сырья', predictions_valid_0.mean(), end='')
print(' | RMSE модели:', rmse_0, '| R2:', r2_0)

Средний запас предсказанного сырья 92.59256778438038 | RMSE модели: 37.5794217150813 | R2: 0.27994321524487786


In [ ]:
mse_1 = mean_squared_error(target_valid_1, predictions_valid_1)
rmse_1 = mse_1 ** 0.5
r2_1 = model_1.score(features_valid_1, target_valid_1)
print('Средний запас предсказанного сырья', predictions_valid_1.mean(), end='')
print(' | RMSE модели:', rmse_1, '| R2:', r2_1)

Средний запас предсказанного сырья 68.728546895446 | RMSE модели: 0.893099286775616 | R2: 0.9996233978805126


In [ ]:
mse_2 = mean_squared_error(target_valid_2, predictions_valid_2)
rmse_2 = mse_2 ** 0.5
r2_2 = model_2.score(features_valid_2, target_valid_2)
print('Средний запас предсказанного сырья', predictions_valid_2.mean(), end='')
print(' | RMSE модели:', rmse_2, '| R2:', r2_2)

Средний запас предсказанного сырья 94.96504596800489 | RMSE модели: 40.02970873393434 | R2: 0.20524758386040443


# Вывод:

Видим, что data_1 дал лучший результат RSME, т.к. значение очень близко к 0, тем самым говорит, что модель хорошо освоила зависимости между признаками. Это можно наблюдать по метрике R2 которая хорошо показывает как хорошо или плохо мы решили задачу. В данном случае "идеальная" модель, R2≈1(0.99).

In [ ]:
'МАЕ:', mean_absolute_error(target_valid_1, predictions_valid_1)

('МАЕ:', 0.718766244212475)

## Подготовка к расчёту прибыли

Рассчитаем достаточный объем сырья для безубыточной разработки новой скважины. Бюджет на разработку 200 скважен = 10 млрд рублей. Одна единица продукта состовляет 450 тыс. рублей. Получаем следующее:

In [ ]:
BEST_WELLS = 200
BUDGET = 10000000000
ONE_PRODUCT_UNIT = 450000

In [ ]:
sufficient_volume_of_raw_materials = (BUDGET / BEST_WELLS) / ONE_PRODUCT_UNIT
print("Достаточный объем для безубычной разработки новой скважины: ",
                                                                sufficient_volume_of_raw_materials)
print('Средний запас сырья в reg_0:', data_0['product'].mean())
print('Средний запас сырья в reg_1:', data_1['product'].mean())
print('Средний запас сырья в reg_2:', data_2['product'].mean())

Достаточный объем для безубычной разработки новой скважины:  111.11111111111111
Средний запас сырья в reg_0: 92.50000000000001
Средний запас сырья в reg_1: 68.82500000000002
Средний запас сырья в reg_2: 95.00000000000004


# Вывод:

По рассчетам видно, что достаточный объем для безубычной разработки новой скважины состовляет 111, т.к. средий запас объема по регионам 70-90. Но это нам ни о чем не говорит, так как есть скважены где объем и больше. Посмотрим на них при построении функции.

Напишем функцию для рассчета прибыли.

In [ ]:
def profit(target, predictions, count=200):
    prob_sorted = predictions.sort_values(ascending=False)
    selected = target[prob_sorted.index][:count]
    return (selected.sum() * ONE_PRODUCT_UNIT) - BUDGET

Подсчет прибыли готов. Рассчитаем прибыль и риски

## Расчёт прибыли и рисков 

Сохраним в переменную для постоянства рандомный вызов и по условию задачи вероятность убытка.

In [ ]:
state = np.random.RandomState(12345)
risk_losses = 2.5

Теперь подсчитем:  
    - среднюю прибыль  
    - 95%-й доверительный интервал  
    - риск убытков  

Так для каждого региона.

In [ ]:
values_0 = []
for i in range(1000):
    target_0_subsample = target_valid_0.sample(500, random_state=state, replace=True)
    probs_0_subsample = prediction_0[target_0_subsample.index]
    values_0.append(profit(target_0_subsample, probs_0_subsample))
    
values_0 = pd.Series(values_0)
mean_profit_0 = values_0.mean()
lower = values_0.quantile(0.025)
upper = values_0.quantile(0.975)
risk_losses_0 = (values_0 < 0).mean()
risk_losses_0 = risk_losses_0 * 100

print('Средняя прибыль geo_0:', mean_profit_0)
print("95%-й доверительный интервал geo_0 =", [lower, upper])
print('Риск потерь от geo_0 =', risk_losses_0, '%')

Средняя прибыль geo_0: 425938526.9105923
95%-й доверительный интервал geo_0 = [-102090094.83793654, 947976353.358369]
Риск потерь от geo_0 = 6.0 %


In [ ]:
values_1 = []
for i in range(1000):
    target_1_subsample = target_valid_1.sample(500, random_state=state, replace=True)
    probs_1_subsample = prediction_1[target_1_subsample.index]
    values_1.append(profit(target_1_subsample, probs_1_subsample))
    
values_1 = pd.Series(values_1)
mean_profit_1 = values_1.mean()
lower = values_1.quantile(0.025)
upper = values_1.quantile(0.975)
risk_losses_1 = (values_1 < 0).mean()
risk_losses_1 = risk_losses_1 * 100

print('Средняя прибыль geo_1:', mean_profit_1)
print("95%-й доверительный интервал geo_1 =", [lower, upper])
print('Риск потерь от geo_1 =', risk_losses_1, '%')

Средняя прибыль geo_1: 518259493.69732493
95%-й доверительный интервал geo_1 = [128123231.43308629, 953612982.0669085]
Риск потерь от geo_1 = 0.3 %


In [ ]:
values_2 = []
for i in range(1000):
    target_2_subsample = target_valid_2.sample(500, random_state=state, replace=True)
    probs_2_subsample = prediction_2[target_2_subsample.index]
    values_2.append(profit(target_2_subsample, probs_2_subsample))
    
values_2 = pd.Series(values_2)
mean_profit_2 = values_2.mean()
lower = values_2.quantile(0.025)
upper = values_2.quantile(0.975)
risk_losses_2 = (values_2 < 0).mean()
risk_losses_2 = risk_losses_2 * 100

print('Средняя прибыль geo_2:', mean_profit_2)
print("95%-й доверительный интервал geo_2 =", [lower, upper])
print('Риск потерь от geo_2 =', risk_losses_2, '%')

Средняя прибыль geo_2: 420194005.34405005
95%-й доверительный интервал geo_2 = [-115852609.16001143, 989629939.844574]
Риск потерь от geo_2 = 6.2 %


In [ ]:
risk_losses_all = pd.DataFrame({'процент риска': [risk_losses_0, risk_losses_1, risk_losses_2],
                                'название региона': ['geo_0', 'geo_1', 'geo_2']}, index=[0, 1, 2])

In [ ]:
for i in risk_losses_all['процент риска']:
    if i < risk_losses:
        print('Регион с риском меньше 2.5%:', 
               risk_losses_all.loc[risk_losses_all['процент риска'] == i, 
                                                'название региона'].values, '=', i, '%')

Регион с риском меньше 2.5%: ['geo_1'] = 0.3 %


# Вывод:

Мой выбор региона для разработки скважин это регион geo_1. Самым главный порогом для принятия решения является риск убытков. У geo_1 риск убытков соотвествует Т3, а значит получить шанс убытков в этом регионе самый маленький, что делает его победителем в выборах для разработки скважен. К тому же средненный доход и доверительный интервал выше чем у других, что говорит о том, что окупаемость и рентабельность этого региона нам подходит для лучше, чем остальные регионы и с большей долей вероятности она нам принесет прибыль, т.к. это единственный регион у которого судя по доверительному интервалу нет отрицательной прибыли.